# Cross-slope section

We compute a cross-slop section of fields through gridded data in ACCESS-OM2-01 using the  [`metpy.interpolate.cross_section` function](https://unidata.github.io/MetPy/latest/examples/cross_section.html#sphx-glr-examples-cross-section-py).

Load modules

In [1]:
import cosima_cookbook as cc
from cosima_cookbook import explore

from dask.distributed import Client
import numpy as np
import xarray as xr

import xgcm

from metpy.interpolate import cross_section

# For plotting
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import cmocean as cm
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import rc
rc('font', **{'size':25})

Start a cluster with multiple cores

In [2]:
client = Client() 
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/35049/status,
Dashboard: /proxy/35049/status,Workers: 8
Total threads: 48,Total memory: 188.55 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36493,Workers: 8
Dashboard: /proxy/35049/status,Total threads: 48
Started: Just now,Total memory: 188.55 GiB
Comm: tcp://127.0.0.1:38467,Total threads: 6
Dashboard: /proxy/44715/status,Memory: 23.57 GiB
Nanny: tcp://127.0.0.1:41439,


Nominate a database from which to load the data and define an experiment

In [3]:
# Load database
session = cc.database.create_session()

In [4]:
depth = 3000

In [5]:
#Pick shelf coordinates
shelf_coord = (-62, -60+100)
deep_coord  = (-56.5, -59+100)


Load velocity and bathymetry data

In [ ]:
# Select data in the southern Southern Ocean
lat_slice = slice(-80, -59)

# We will use an annual average from one year in the RYF run
expt = '01deg_jra55v13_ryf9091'
start_time = '1950-01-31 00:00:00'
end_time = '1959-12-31 00:00:00'

# Import bathymetry
hu = cc.querying.getvar(expt, 'hu', session, n=1)
hu = hu.sel(yu_ocean=lat_slice)
hu = hu.load()


In [ ]:
## importing density - ignore warnings
pot_rho_2 = cc.querying.getvar(expt, 'pot_rho_2', session,
                               start_time=start_time, end_time=end_time, frequency='1 monthly',
                               attrs={'cell_methods': 'time: mean'},
                               chunks={})
pot_rho_2

In [ ]:
#importing potential temperature - ignore warnings
pot_temp =cc.querying.getvar(expt=expt, variable='pot_temp', 
                             session=session, frequency='1 monthly',
                             attrs={'cell_methods': 'time: mean'},
                             start_time=start_time, end_time=end_time,
                             chunks={})

In [ ]:
# Load model grid information with depth
path_to_folder = '/g/data/ik11/outputs/access-om2-01/01deg_jra55v13_ryf9091/output000/ocean/'
grid = xr.open_mfdataset(path_to_folder+'ocean_grid.nc', combine='by_coords')

# Give information on the grid: location of u (momentum) and t (tracer) points on B-grid 
ds = xr.merge([pot_temp.sel(yt_ocean=lat_slice).sel(st_ocean=slice(0, depth)), grid.sel(yu_ocean=lat_slice).sel(yt_ocean=lat_slice)])
ds.coords['xt_ocean'].attrs.update(axis='X')
ds.coords['xu_ocean'].attrs.update(axis='X', c_grid_axis_shift=0.5)
ds.coords['yt_ocean'].attrs.update(axis='Y')
ds.coords['yu_ocean'].attrs.update(axis='Y', c_grid_axis_shift=0.5)

grid_depth = xgcm.Grid(ds, periodic=['X'])
grid_depth

Take time mean for density and temperature

In [ ]:
pot_rho_2 = pot_rho_2.mean(dim = 'time')
pot_rho_2 = pot_rho_2.sel(yt_ocean = lat_slice).sel(st_ocean=slice(0,depth))
pot_rho_2

In [ ]:
pot_temp = pot_temp.mean(dim = 'time')
pot_temp = pot_temp.sel(yt_ocean = lat_slice).sel(st_ocean=slice(0,depth))
pot_temp

In [ ]:
#choosing number of steps in cross section
step_no = 400

# interpolation on to u grid
#pot_rho_2 = grid_depth.interp(grid_depth.interp(pot_rho_2, axis = 'X').chunk({'yt_ocean': 500}) , axis = 'Y', boundary = 'extend')#.sel(yu_ocean=lat_slice).sel(st_ocean=slice(0,depth))

# Create dataset
ds_pot_rho_2 = xr.Dataset({"pot_rho_2": pot_rho_2, "lat": pot_rho_2.yt_ocean, "lon": pot_rho_2.xt_ocean})

# Interpolate to xu_ocean and yu_ocean
# Rename coordinate names
ds_pot_rho_2 = ds_pot_rho_2.rename({'xt_ocean': 'x', 'yt_ocean': 'y'})

# Convert latitude from ACCESS-OM2 default range of [-280, 80] to [-180, 180] which is what metpy expects.
ds_pot_rho_2['x'] = ds_pot_rho_2['x'] + 100
ds_pot_rho_2['x'].attrs = pot_rho_2['xt_ocean'].attrs

# MetPy parsing
pot_rho_2_parsed = ds_pot_rho_2.metpy.parse_cf('pot_rho_2', coordinates={'y': 'y', 'x': 'x'})
pot_rho_2_section = cross_section(pot_rho_2_parsed, start=(shelf_coord[0], shelf_coord[1]), end=(deep_coord[0], deep_coord[1]), steps = step_no, interp_type='linear')

In [ ]:

# Create dataset
ds_pot_temp= xr.Dataset({"pot_temp": pot_temp, "lat": pot_temp.yt_ocean, "lon": pot_temp.xt_ocean})
# Interpolate to xu_ocean and yu_ocean
# Rename coordinate names
ds_pot_temp = ds_pot_temp.rename({'xt_ocean': 'x', 'yt_ocean': 'y'})

# Convert longitude from ACCESS-OM2 range of [-280, 80] to [-180, 180] which is what metpy expects.
ds_pot_temp['x'] = ds_pot_temp['x'] + 100

ds_pot_temp['x'].attrs = pot_temp['xt_ocean'].attrs

# MetPy parsing
pot_temp_parsed = ds_pot_temp.metpy.parse_cf('pot_temp', coordinates={'y': 'y', 'x': 'x'})
pot_temp_section = cross_section(pot_temp_parsed,
                                 start = (shelf_coord[0], shelf_coord[1]),
                                 end = (deep_coord[0], deep_coord[1]),
                                 steps = step_no,
                                 interp_type = 'linear')

In [ ]:
# load variables
pot_temp_section.load()
pot_rho_2_section.load()

Finally calculate the distance along the transect (for plotting purposes)

In [ ]:
# Define number of points you want to interpolate
step_no = 400
# Radius of the Earth in km
r = 6371

# Difference between points in lat/lon space
dlon = deep_coord[1] - shelf_coord[1]
dlat = deep_coord[0] - shelf_coord[0]

# Calculate distance in km between the two end points
distance_endpoints = r * np.deg2rad(np.sqrt(dlat**2 + (dlon * np.cos(np.deg2rad(np.mean([shelf_coord[0], deep_coord[0]]))))**2))

# Create array with length of step_no
distance_in_km = np.linspace(0, distance_endpoints, step_no)

# Repeat by the number of depth levels
distance_in_km = np.tile(distance_in_km, (len(pot_temp_section.st_ocean), 1))

## Plotting cross-slope section of potential temperature

In [ ]:
fig, axs = plt.subplots( figsize=(10, 15), sharex = True)
ft_size = 16

axs.set_title('Cross-slope section of \n (%.1f,%.1f) to (%.1f,%.1f)' % (shelf_coord[0], shelf_coord[1]-100, deep_coord[0], deep_coord[1]-100), loc='left')

cmesh = axs.pcolormesh(distance_in_km, pot_temp_section.st_ocean.sel(st_ocean=slice(0, 5000)).values, pot_temp_section.sel(st_ocean=slice(0,5000)).values, cmap = 'RdBu_r')

# Colorbar
cbar = plt.colorbar(cmesh, orientation = 'horizontal')
cbar.set_label(r'$\theta$ / K')
#cbar.ax.tick_params(labelsize=ft_size)

cs = axs.contour(distance_in_km[0, :], pot_rho_2_section.st_ocean, pot_rho_2_section,
                 cmap = 'gray')
axs.clabel(cs, cs.levels, fontsize = 12,colors = 'k', inline=True, inline_spacing=3)

# Axes
plt.gca().invert_yaxis()
plt.xlim([0, 350])
plt.xlabel('Distance (km)')
plt.ylabel('Depth (m)')
axs.xaxis.set_minor_locator(MultipleLocator(10))
axs.yaxis.set_minor_locator(MultipleLocator(50))
plt.tight_layout();